1. есть сайт Зак.Собрания НО - https://www.zsno.ru/law/bazovye-zakony-nizhegorodskoy-oblasti/?PAGEN_1=2
2. надо найти ссылки на законы на этой странице, они вида http://pravo.gov.ru/proxy/ips/?docbody=&amp;link_id=22&amp;nd=152179092&amp;intelsearch=
3. взять айди документа 152179092
4. подставить в ссылку вида http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=152249630&page=1&rdk=0&link_id=180 , заменив айди
5. извлечь инфу из документа:
а) дата подписания (первая дата)
б) дата принятия (вторая дата)
в) название документа
г) номер документа
6. сложить в список словариков, показать как табличку датафрейма

In [4]:
import re
import requests
import time
import pandas as pd
import logging
from datetime import datetime

In [5]:
today = datetime.now()
date = today.strftime('%Y-%m-%d')

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0'
headers = {'User-Agent': user_agent}

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)
        
handler = logging.StreamHandler()
FORMAT = '%(asctime)s %(levelname)-8s %(message)s'
formatter = logging.Formatter(fmt=FORMAT)
handler.setFormatter(formatter)
logger.addHandler(handler)

In [8]:
class Parser:
    
    def __init__(self):
        self.result = []
        
    def get_html(self, url, encoding='utf-8'):
        time.sleep(1)
        response = requests.get(url, headers=headers)
        assert response.status_code == 200, 'Статус респонса не 200'
        html = response.content.decode(encoding)
        return html

In [7]:
class ZSParser(Parser):

    base_url = 'https://www.zsno.ru/law/bazovye-zakony-nizhegorodskoy-oblasti/'
    page_url = base_url + '?PAGEN_1='
    doc_url = 'http://pravo.gov.ru/proxy/ips/?doc_itself=&nd={_id}&page={page}&rdk=1&link_id={link_id}'
    pattern_sign_date_1 = re.compile(r'<p class="a">Нижний Новгород</p>\s+<p class="a">([^<]+)')
    pattern_sign_date_2 = r'<p class="a" style="text-indent:18.75pt">Нижний Новгород</p>\s+<p class="a" style="text-indent:18.75pt">([^<]+)'
    pattern_sign_date_3 = r'<p class="a" style="text-align:justify">Нижний Новгород</p>\s+<p class="a" style="text-align:justify">([^<]+)'
    pattern_num_1 = r'<p class="a">№([^<]+)'
    pattern_num_2 = r'<p class="a" style="text-indent:18.75pt">№([^<]+)'
    pattern_num_3 = r'<p class="a" style="text-align:justify">№([^<]+)'
    pattern_name_1 = r'<span style="color:#0D0D0D">О([^a-zA-Z<&]+)'
    pattern_name_2 = r'<p class="a0" align="center" style="text-align:center">([^a-zA-Z<&]+)'
    pattern_name_3 = r'<p class="a" align="center" style="text-align:center"><b>([^a-zA-Z<&]+)'
    pattern_name_4 = r'<pre>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ([^a-zA-Z<&]+)'
    pattern_blocks = r'<a href="http://pravo\.gov\.ru/proxy/ips/\?docbody=\&.{10}[^"]+'
    pattern_id = r'&link_id=(\d+)&nd=(\d+)'
    pattern_page = r'class="p_nav__page[^\d]+(\d+)'
    
    def __init__(self, max_pages=None):
        super().__init__()
        self.max_pages = max_pages

    def parse(self):
        html = super().get_html(self.base_url)
        if self.max_pages:
            end_page = self.max_pages + 1
        else:
            pages = re.findall(self.pattern_page, html)
            end_page = int(pages[-1]) + 1
        for page in range(1, end_page):
            self.parse_page(page)
        df = pd.DataFrame(self.result)
        df = df.sort_values(by='id')
        df_date = df[df['sign_date'].notna()]
        df_no_date = df[df['sign_date'].isna()]
        df_date.to_excel('ZS_' + date +'_date.xlsx', index=False)
        df_no_date.to_excel('ZS_'+date+'_no_date.xlsx', index=False)
        return df

    def parse_page(self, page):
        logger.info(f'page: {page}')
        url = self.page_url + str(page)
        html = self.get_html(url)
        blocks = re.findall(self.pattern_blocks, html)

        for block in blocks:
            link_id, _id = re.search(self.pattern_id, block).groups()
            url = self.doc_url.format(_id=_id, page=page, link_id=link_id)
            self.get_doc(_id, url)

    def get_doc(self, _id, url):
        logger.info(f'doc: {_id}')
        html = super().get_html(url, 'windows-1251')
        sign_date = self.pattern_sign_date_1.search(html)
        if not sign_date:
            sign_date = re.search(self.pattern_sign_date_2, html)
        if not sign_date:
            sign_date = re.search(self.pattern_sign_date_3, html)
        num = re.search(self.pattern_num_1, html)
        if not num:
            num = re.search(self.pattern_num_2, html)
        if not num:
            num = re.search(self.pattern_num_3, html)
        name = []
        name = ['О'] + re.findall(self.pattern_name_1, html)
        if name == ['О']:
            name = re.findall(self.pattern_name_2, html)
        if not name:
            name = re.findall(self.pattern_name_3, html)
        if not name:
            name = re.findall(self.pattern_name_4, html)
        s = ''
        for j in name:
            if j != 'НИЖЕГОРОДСКАЯ ОБЛАСТЬ' and j != 'ЗАКОН' and (not '№' in j):
                s += j
        row = {
            'id': _id,
            'sign_date': sign_date.groups()[0].strip() if sign_date else None,
            'num': num.groups()[0].strip() if num else None,
            'name': s.strip().replace('\r', ' ') or None,
            'url': url,
        }
        self.result.append(row)

In [9]:
parser = Parser_ZS(max_pages=2)
res = parser.parse()
res

2025-02-17 19:13:13,518 INFO     page: 1
2025-02-17 19:13:14,902 INFO     doc: 152251792
2025-02-17 19:13:16,313 INFO     doc: 152249630
2025-02-17 19:13:20,810 INFO     doc: 152247253
2025-02-17 19:13:22,209 INFO     doc: 152245252
2025-02-17 19:13:26,090 INFO     doc: 152240962
2025-02-17 19:13:28,432 INFO     doc: 152244328
2025-02-17 19:13:29,852 INFO     doc: 152175447
2025-02-17 19:13:31,977 INFO     doc: 152224839
2025-02-17 19:13:33,402 INFO     doc: 152224849
2025-02-17 19:13:35,787 INFO     doc: 152222338
2025-02-17 19:13:38,258 INFO     page: 2
2025-02-17 19:13:39,597 INFO     doc: 152220670
2025-02-17 19:13:41,093 INFO     doc: 152219459
2025-02-17 19:13:44,327 INFO     doc: 152189555
2025-02-17 19:13:46,952 INFO     doc: 152185445
2025-02-17 19:13:49,702 INFO     doc: 152185446
2025-02-17 19:13:52,407 INFO     doc: 152185450
2025-02-17 19:13:54,672 INFO     doc: 152178211
2025-02-17 19:13:56,512 INFO     doc: 152179092
2025-02-17 19:13:57,956 INFO     doc: 152168863
2025-0

,id,sign_date,num,name,url
18,152168863,7 ноября 2022 года,155-З,О специализированных \nорганизациях для оказан...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
19,152168874,7 ноября 2022 года,157-З,О заготовке лесных ресурсов \nгражданами для с...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
6,152175447,14 декабря 2023 года,143-З,Об \nисключительных случаях заготовки,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
16,152178211,2 декабря 2022 года,172-З,Об утверждении методики \nраспределения субвен...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
17,152179092,2 декабря 2022 года,179-З,О единовременной денежной \nвыплате супругамв ...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
13,152185445,15 декабря 2022 года,194-З,О Ветеранах труда и \nВетеранах трудаНижегород...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
14,152185446,15 декабря 2022 года,195-З,О мерах социальной поддержки \nинвалидов боевы...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
15,152185450,15 декабря 2022 года,196-З,О сохранении на территории \nНижегородской обл...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
12,152189555,27 декабря 2022 года,209-З,Об особенностях развития \nэкологического тури...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...
11,152219459,2 августа 2023 года,95-З,О Торгово-промышленной \nпалате Нижегородской ...,http://pravo.gov.ru/proxy/ips/?doc_itself=&nd=...


In [11]:
class DOCParser(Parser):

    base_url = 'https://ballotpedia.org'
    doc_url = 'https://ballotpedia.org/Supreme_Court_cases,_October_term_2024-2025'
    pattern_circuit = r'_Circuit">([^<]+)'
    pattern_term = r'<b>Term:</b>([^<]+)'
    pattern_date = r'Important dates</b></td></tr><tr><td colspan="2" style="text-align:center;">([^<]+)'
    pattern_num = r'<b>Docket number:</b> ([^<]+)'
    pattern_urls = r'<i><a href="([^"]+)'
    pattern_name = r'<table class="infobox".+<big><i><b>([^<]+)'
    pattern_blocks = r'id="[^"]+_Circuit">.+?</ul>'
    pattern_petitioner = r'<li><b>Petitioner</b>:([^<]+)'
    pattern_respondent = r'<li><b>Respondent</b>:([^<]+)'
    
    def __init__(self):
        super().__init__()

    def parse(self):
        html = super().get_html(self.doc_url)
        blocks = re.findall(self.pattern_blocks, html, re.DOTALL)
        for block in blocks:
            urls = re.findall(self.pattern_urls, block)
            circuit = re.findall(self.pattern_circuit, block)
            for u in urls:
                time.sleep(2)
                url = self.base_url + u
                logger.info(f'circuit: {circuit}, url: {url}')
                self.parse_doc(url, circuit)
        df = pd.DataFrame(self.result)
        return df

    def parse_doc(self, url, circuit):
        html = super().get_html(url)
        name = re.findall(self.pattern_name, html, re.DOTALL)
        num = re.findall(self.pattern_num, html, re.DOTALL)
        term = re.findall(self.pattern_term, html, re.DOTALL)
        date = re.findall(self.pattern_date, html, re.DOTALL)
        petitioner = re.findall(self.pattern_petitioner, html, re.DOTALL)
        respondent = re.findall(self.pattern_respondent, html, re.DOTALL)
        row = {
            'circuit': circuit[0], 
            'name':  name[0] if name else None, 
            'num': num[0] if num else None, 
            'term': term[0] if term else None,
            'date': date[0] if date else None,
            'petitioner': petitioner[0] if petitioner else None,
            'respondent': respondent[0] if respondent else None,
        }
        self.result.append(row)

In [13]:
parser = Parser_Doc()
res = parser.parse()
res

2025-02-17 19:15:13,389 INFO     circuit: ['1st Circuit'], url: https://ballotpedia.org/Smith_%26_Wesson_Brands_v._Estados_Unidos_Mexicanos
2025-02-17 19:15:18,060 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Medical_Marijuana,_Inc._v._Horn
2025-02-17 19:15:22,745 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Delligatti_v._United_States
2025-02-17 19:15:27,220 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Cunningham_v._Cornell_University
2025-02-17 19:15:31,921 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/BLOM_Bank_SAL_v._Honickman
2025-02-17 19:15:36,636 INFO     circuit: ['2nd Circuit'], url: https://ballotpedia.org/Fuld_v._Palestine_Liberation_Organization
2025-02-17 19:15:41,165 INFO     circuit: ['3rd Circuit'], url: https://ballotpedia.org/Kousisis_v._United_States
2025-02-17 19:15:45,914 INFO     circuit: ['3rd Circuit'], url: https://ballotpedia.org/Commissioner_of_Internal_Revenue_v._Zuch
2025-02-17 19:1

,circuit,name,num,term,date,petitioner,respondent
0,1st Circuit,Smith &amp; Wesson Brands v. Estados Unidos Me...,23-1141,2024,"Argument: March 4, 2025","Smith &amp; Wesson Brands, Inc.,",Estados Unidos Mexicanos\n
1,2nd Circuit,"Medical Marijuana, Inc. v. Horn",23-365,2024,"Argued: October 15, 2024","Medical Marijuana, Inc., et al.\n",Douglas J. Horn\n
2,2nd Circuit,Delligatti v. United States,23-825,2024,"Argued: November 12, 2024",Salvatore Delligatti\n,United States\n
3,2nd Circuit,Cunningham v. Cornell University,23-1007,2024,"Argued: January 22, 2025",Casey Cunningham\n,Cornell University\n
4,2nd Circuit,BLOM Bank SAL v. Honickman,23-1259,2024,"Argument: March 3, 2025",BLOM Bank SAL\n,"Michal Honickman,"
5,2nd Circuit,Fuld v. Palestine Liberation Organization,24-20,2024,Pending,"Miriam Fuld, et al.\n","Palestine Liberation Organization, et al.\n"
6,3rd Circuit,Kousisis v. United States,23-909,2024,"Argued: December 9, 2024",None,None
7,3rd Circuit,Commissioner of Internal Revenue v. Zuch,24-416,2024,Pending,Commissioner of Internal Revenue\n,Jennifer Zuch\n
8,4th Circuit,Lackey v. Stinnie,23-621,2024,"Argued: October 8, 2024","Gerald F. Lackey, in His Official Capacity as...","Damian Stinnie, et al.\n"
9,4th Circuit,"E.M.D. Sales, Inc. v. Carrera",None,2024,None,None,None


In [2]:
from zsno import ZSParser

ZSParser.parse()

ModuleNotFoundError: No module named 'zsno'